<a href="https://colab.research.google.com/github/diem-ai/datascience-projects/blob/master/twitter_bot/ebay_bot_w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Setup

In [0]:
from google.colab import drive
drive.mount('/content/drive')
import sys
# To add a directory with your code into a list of directories 
# which will be searched for packages
sys.path.append('/content/drive/My Drive/Colab Notebooks')

!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Install packages

In [0]:
! pip install --upgrade gensim
! pip install --upgrade emoji

     |████████████████████████████████| 24.2MB 98kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=c7f22f229e1a973a737baacacf60c86c078c256d1b662e51f98a5f025a15fa11
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


# Libraries

In [0]:
import pandas as pd
import math
import re
import nltk
import string

#Download only once
nltk.download('punkt')  # pre-trained tokenizer for English
nltk.download('wordnet') # lexical database for the English language
nltk.download('stopwords')

from gensim.test.utils import common_texts
from gensim.corpora import Dictionary
from gensim.models import Word2Vec , WordEmbeddingSimilarityIndex
from gensim.similarities import SoftCosineSimilarity, SparseTermSimilarityMatrix
from gensim.models import TfidfModel
from multiprocessing import cpu_count


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Data Preparation 
Load twitter dataset into pandas dataframe

In [0]:
filepath = '/content/drive/My Drive/data/' 

df = pd.read_csv(filepath + "twcs.csv.zip")

df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


Retrive list of companies providing customer support on twitter

In [0]:
pattern = r"\A[a-z]+\Z"

authors = []
authors = [ val for val in df["author_id"].values if (re.search(pattern, val.lower())) ]

# remove duplicated values in the list
authors = list(set(authors))

#print(re.search(pattern, "12345"))

Select eBay's data

In [0]:
select_author =  "AskeBay"

data = df [df["author_id"] == select_author]

data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
683,1224,AskeBay,False,Wed Nov 22 09:24:40 +0000 2017,@115930 Hi Simon! I’m sorry to hear this! Soun...,1225,1226.0
685,1227,AskeBay,False,Wed Nov 22 11:48:52 +0000 2017,@115930 Thanks for the update Simon! Please le...,NaN,1225.0
704,1245,AskeBay,False,Wed Nov 22 08:40:37 +0000 2017,@115936 Hi. Sorry to hear the buyers have not ...,NaN,1246.0
706,1247,AskeBay,False,Wed Nov 22 08:37:12 +0000 2017,@115938 Hi there. Sorry to hear the item has n...,1248,1249.0
709,1251,AskeBay,False,Wed Nov 22 09:25:52 +0000 2017,@115938 As the seller is responsible for the c...,1252,1250.0


Select neccessary features for data processing

In [0]:
# select sprintcare's responses to their customers questions

select_features = ["tweet_id", "text",	"response_tweet_id",	"in_response_to_tweet_id"]

data = data[select_features]

data.head()

,tweet_id,text,response_tweet_id,in_response_to_tweet_id
683,1224,@115930 Hi Simon! I’m sorry to hear this! Soun...,1225,1226.0
685,1227,@115930 Thanks for the update Simon! Please le...,NaN,1225.0
704,1245,@115936 Hi. Sorry to hear the buyers have not ...,NaN,1246.0
706,1247,@115938 Hi there. Sorry to hear the item has n...,1248,1249.0
709,1251,@115938 As the seller is responsible for the c...,1252,1250.0


Create a data frame with all responses from eBay

In [0]:
# select eBay's questions 

questions = df[ df["text"].str.contains("AskeBay") ]

questions.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
684,1225,115930,True,Wed Nov 22 10:01:38 +0000 2017,@AskeBay sure will do. Getting refunded by the...,1227,1224.0
705,1246,115936,True,Tue Nov 21 21:24:33 +0000 2017,EVERYTIME i sell something on @115937 @AskeBay...,1245,NaN
707,1248,115938,True,Wed Nov 22 08:40:46 +0000 2017,@AskeBay The item has been marked as delivered...,1250,1247.0
708,1250,115938,True,Wed Nov 22 08:41:35 +0000 2017,@AskeBay Need to contact customer service dire...,1251,1248.0
710,1252,115938,True,Wed Nov 22 10:29:17 +0000 2017,"@AskeBay Ok, thank you for your help, I'll do ...",1253,1251.0


In [0]:
len(questions)

8166

In [0]:
questions.iloc[0]["text"]

'@AskeBay sure will do. Getting refunded by the sellers in the meantime.'

In [0]:
idx = 0
print("id: {}, question: {}".format(questions.iloc[idx]["author_id"]
                                    , questions.iloc[idx]["text"]))

print("answer: {}".format(data.iloc[idx]["text"]))

id: 115930, question: @AskeBay sure will do. Getting refunded by the sellers in the meantime.
answer: @115930 Hi Simon! I’m sorry to hear this! Sounds like your account may have been compromised. I strongly advise you to contact our security teams &gt; https://t.co/hV7XdVcjA7 if you haven’t already done so, to make sure you do to ensure your account is fully secured. ^GR


In [0]:
idx = 100
print("id: {}, question: {}".format(questions.iloc[idx]["author_id"]
                                    , questions.iloc[idx]["text"]))

print("answer: {}".format(data.iloc[idx]["text"]))

id: 123601, question: @AskeBay Well could you check my account status then please and see if I'm due you any payment? Think I cleared it earlier.
answer: @123128 Thanks for the info, Ethan! We issue selling limits on all accounts (they renew every 30-days). Take a look at https://t.co/m4mQJyZjIW for more information and how you can request higher limits. Let us know if you have further questions. Thanks! ^AE


In [0]:
idx = 500
print("id: {}, question: {}".format(questions.iloc[idx]["author_id"]
                                    , questions.iloc[idx]["text"]))

print("answer: {}".format(data.iloc[idx]["text"]))

id: 156038, question: @AskeBay please respond to my DM
answer: @154436 If you accepted the return &amp; they haven't returned it in 14 days (if the case was opened on the UK site), we can close the case for you. ^BL


In [0]:
def get_tweet_from_id(data, tweet_id):
  """
  @data: [pandas dataframe] : pandas dataframe
  @tweet_id: [integer] : an tweet ID
  @return : [list] : a tweet corresponding with tweet_id input
  """
  return data[data["tweet_id"] == tweet_id]["text"].values

Create a pair of list of questions and responses

In [0]:
ebay_questions = []
ebay_responses = []

for i in range(len(questions)):
  tweet = questions.iloc[i]
  
  text = tweet["text"]
  owner = tweet["author_id"]
  ebay_questions.append(owner + "-" + text)

  rep_ids = tweet["response_tweet_id"]
  if (rep_ids and (not pd.isna(rep_ids))):
    if "," in rep_ids:
      rep_ids = rep_ids.split(",")[0]
    
    # convert an array into string
    response = "".join( get_tweet_from_id(df, int(rep_ids)) )
    ebay_responses.append(response)
  else:
        ebay_responses.append("")

In [0]:
ebay_questions[2]

'115938-@AskeBay The item has been marked as delivered, and the system would not let me contact ebay customer service directly. It was not sellers fault, it was mismanaged somewhere between shipment and delivery. Still file a claim?'

In [0]:
ebay_responses[2]

'@AskeBay Need to contact customer service directly. Please DM contact email for customer service'

# Data Processing

In [0]:
import spacy
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
stops = spacy.lang.en.stop_words.STOP_WORDS
from nltk.corpus import stopwords

SW =  set(stopwords.words("english"))
SW.add("askebay")

def normalize(text):
  """
  Lemmatize tokens and remove them if they are stop words
  """
  text = nlp(text.lower())
  lemmatized = list()
  for word in text:
    lemma = word.lemma_.strip()
    if (lemma and lemma not in SW ):
      lemmatized.append(lemma)
  return lemmatized


# test normalize()
text = ["applesupport tried resetting my settings  restarting my phone  all that 🙃"
, "i need answers because it is annoying 🙃"
, "That's great it has iOS 111 as we can rule out being outdated Any steps tried since this started Do you recall when it started"]

print([normalize(sent) for sent in text])


[['applesupport', 'try', 'reset', 'setting', 'restart', 'phone', '🙃'], ['need', 'answer', 'annoy', '🙃'], ['great', 'ios', '111', 'rule', 'outdate', 'step', 'try', 'since', 'start', 'recall', 'start']]


In [0]:
import emoji

def clean_emoji(text):
  if (emoji.emoji_count(text) > 0):
    return re.sub(emoji.get_emoji_regexp(), r"", text)
  else:
    return text

# test clean_emoji()
text = ["applesupport tried resetting my settings  restarting my phone  all that 🙃"
, "i️ need answers because it is annoying 🙃"
, "That's great it has iOS 111 as we can rule out being outdated Any steps tried since this started Do you recall when it started"]

print([clean_emoji(sent) for sent in text])

print([normalize(clean_emoji(sent)) for sent in text])


['applesupport tried resetting my settings  restarting my phone  all that ', 'i️ need answers because it is annoying ', "That's great it has iOS 111 as we can rule out being outdated Any steps tried since this started Do you recall when it started"]
[['applesupport', 'try', 'reset', 'setting', 'restart', 'phone'], ['i️', 'need', 'answer', 'annoy'], ['great', 'ios', '111', 'rule', 'outdate', 'step', 'try', 'since', 'start', 'recall', 'start']]


In [0]:
# from string import maketrans 
# Remove the punctuation of text
def clean_punctuation(text):
  """
  @text: is a text with/without the punctuation
  @return a text without punctuation
  """
  re_punc = re.compile("[%s]" % re.escape(string.punctuation))
  return re_punc.sub("", text)


# test clean_punctuation()
text = ["applesupport tried resetting my settings #@ restarting my phone && all that 🙃"
, "i️ need answers because it is annoying 🙃 ..."
, "That's great it has iOS 111 as we can rule out being outdated Any steps tried since this started Do you recall when it started"]

print([ clean_punctuation( sent ) for sent in text])

print([ normalize(clean_emoji(clean_punctuation(sent))) for sent in text])

['applesupport tried resetting my settings  restarting my phone  all that 🙃', 'i️ need answers because it is annoying 🙃 ', 'Thats great it has iOS 111 as we can rule out being outdated Any steps tried since this started Do you recall when it started']
[['applesupport', 'try', 'reset', 'setting', 'restart', 'phone'], ['i️', 'need', 'answer', 'annoy'], ['great', 'ios', '111', 'rule', 'outdate', 'step', 'try', 'since', 'start', 'recall', 'start']]


In [0]:
def decontracted(text):
    # specific
    text = re.sub(r"won’t", "will not", text)
    text = re.sub(r"won\'t", "will not", text)

    text = re.sub(r"can’t", "can not", text)
    text = re.sub(r"can\'t", "can not", text)

    text = re.sub(r"dont’t", "do not", text)
    text = re.sub(r"don\'t", "do not", text)

    text = re.sub(r"doesn’t", "does not", text)
    text = re.sub(r"doesn\'t", "does not", text)

    text = re.sub(r"y’all", "you all", text)
    text = re.sub(r"y\'all", "you all", text)
    # general
    text = re.sub(r"’t", " not", text)
    text = re.sub(r"n\'t", " not", text)

    text = re.sub(r"’re", " are", text)
    text = re.sub(r"\'re", " are", text)

    text = re.sub(r"’s", " is", text)
    text = re.sub(r"\'s", " is", text)

    text = re.sub(r"’d", " would", text)
    text = re.sub(r"\'d", " would", text)

    text = re.sub(r"’ll", " will", text)
    text = re.sub(r"\'ll", " will", text)

    text = re.sub(r"’t", " not", text)
    text = re.sub(r"\'t", " not", text)

    text = re.sub(r"’ve", " have", text)
    text = re.sub(r"\'ve", " have", text)

    text = re.sub(r"’m", " am", text)
    text = re.sub(r"\'m", " am", text)
    return text

text = "@applesupport y’all gotta fix this i.t problem on everyone is phone i just want to be able to say it again not i.t',"
print(decontracted(text))

print(normalize(clean_emoji(clean_punctuation(decontracted(text )))))

@applesupport you all gotta fix this i.t problem on everyone is phone i just want to be able to say it again not i.t',
['applesupport', 'get', 'fix', 'problem', 'everyone', 'phone', 'want', 'able', 'say']


In [0]:
def clean_num(text):
  """
  @Remove numberics in the text
  @return the text without numberics
  """
  re_num = r"\d+"
  return re.sub(re_num, "", text)

# test clean_num()
text = "115855 That's great it has iOS 11.1 as we can rule out being outdated. Any steps tried since this started?"
text = clean_punctuation(text)
print( clean_num(text) )


 Thats great it has iOS  as we can rule out being outdated Any steps tried since this started


In [0]:
def preprocessing(text):
#  text = clean_at(text)
  text = clean_emoji(text)
  text = decontracted(text)
  text = clean_punctuation(text)
  text = clean_num(text)
  text = normalize(text)
  return text

# test preprocessing()
text = ["@applesupport tried resetting my settings .. restarting my phone .. all that"
      , "i need answers because it is annoying 🙃"
      , "@115855 That's great it has iOS 11.1 as we can rule out being outdated. Any steps tried since this started?"]
print( [preprocessing(sent) for sent in text] )

[['applesupport', 'try', 'reset', 'setting', 'restart', 'phone'], ['need', 'answer', 'annoy'], ['great', 'ios', 'rule', 'outdate', 'step', 'try', 'since', 'start']]


In [0]:
def clean_at(text):
  re_at = r"^@\S+"
  return re.sub(re_at, "", text)

# test clean_at()
text_at = "@115855 That's great it has iOS 11.1 as we can rule out being outdated. Any steps tried since this started?"
clean_at(text_at)

" That's great it has iOS 11.1 as we can rule out being outdated. Any steps tried since this started?"

In [0]:
# processing questions
processed_ebay_questions = [preprocessing(quest) for quest in ebay_questions]
ebay_responses = [clean_at(resp) for resp in ebay_responses]

In [0]:
processed_ebay_questions[:2]

[['sure', 'get', 'refund', 'seller', 'meantime'],
 ['everytime',
  'sell',
  'something',
  'bid',
  'come',
  'back',
  'fraudulent',
  'repost',
  'day',
  'waste',
  'ugh']]

In [0]:
ebay_questions[:2]

['115930-@AskeBay sure will do. Getting refunded by the sellers in the meantime.',
 '115936-EVERYTIME i sell something on @115937 @AskeBay the bid comes back as fraudulent and i have to repost.  7 days wasted.  I’m done.  Ugh.']

In [0]:
ebay_responses[:2]

[' Thanks for the update Simon! Please let us know if there’s anything we can help you with! Have a great day! ^GR',
 ' Hi. Sorry to hear the buyers have not followed through. Did you open an unpaid item claim? You can set buyer requirements which will block buyers who have strikes against them which will reduce this from happening. See here&gt; https://t.co/5GCQb0IEA0. Thanks ^SG']

In [0]:
processed_ebay_questions[3:5]

[['need',
  'contact',
  'customer',
  'service',
  'directly',
  'please',
  'dm',
  'contact',
  'email',
  'customer',
  'service'],
 ['okay', 'thank', 'help']]

In [0]:
ebay_responses[3:5]

[' As the seller is responsible for the courier, they can claim for lost items from the courier. You still have to open a claim through eBay to be protected. Please follow the previous steps. We cant outbound DM. You can DM us if you have any other questions. ^SG',
 " You're more than welcome. Any other questions, please dont hesitate to ask. Thanks ^SG"]

# Build model with small dataset

In [0]:
## make a test on 10 questions and their 10 responses

test_questions = processed_ebay_questions[:10]

test_responses = ebay_responses[0:10]

test_dictionary = Dictionary(test_questions)

test_tfidf = TfidfModel(dictionary = test_dictionary)

test_word2vec = Word2Vec(test_questions
                     , workers=cpu_count()
                     , min_count=5
                     , size=300
                     , seed=12345)

test_sim_index = WordEmbeddingSimilarityIndex(test_word2vec.wv)

test_sim_matrix = SparseTermSimilarityMatrix(test_sim_index
                                               , test_dictionary
                                               , test_tfidf
                                               , nonzero_limit=100)
test_bow_corpus = [test_dictionary.doc2bow(document) for document in test_questions]

test_docsim_index = SoftCosineSimilarity(test_bow_corpus, test_sim_matrix, num_best=5)


# Make a test with small model

In [0]:
query = "I need to contact customer service to report an purchase issue"

pro_query = preprocessing(query)

test_vector = test_dictionary.doc2bow(pro_query)

test_vector_tfidf = test_tfidf[test_vector]

sims = test_docsim_index[test_vector_tfidf]

for i, score in sims:
  print(ebay_responses[i], score)

 As the seller is responsible for the courier, they can claim for lost items from the courier. You still have to open a claim through eBay to be protected. Please follow the previous steps. We cant outbound DM. You can DM us if you have any other questions. ^SG 0.9561439156532288
 Need to contact customer service directly. Please DM contact email for customer service 0.842599630355835
 0.842599630355835
 Hi there. Sorry to hear the item has not arrived. Is there tracking for the item? If not, I'd advise opening an item not received claim. You can learn more about that here&gt; https://t.co/0toc8cKSB0. Any other questions, let us know. ^SG 0.6582105755805969
 0.52857506275177


/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))


@115938 As the seller is responsible for the courier, they can claim for lost items from the courier. You still have to open a claim through eBay to be protected. Please follow the previous steps. We cant outbound DM. You can DM us if you have any other questions. ^SG 0.9561439156532288

@AskeBay Need to contact customer service directly. Please DM contact email for customer service 0.842599630355835
 0.842599630355835

@115938 Hi there. Sorry to hear the item has not arrived. Is there tracking for the item? If not, I'd advise opening an item not received claim. You can learn more about that here&gt; https://t.co/0toc8cKSB0. Any other questions, let us know. ^SG 0.6582105755805969

 0.52857506275177
/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide



In [0]:
query = "I need an assistance. The item is marked delivery but I never receive it"

pro_query = preprocessing(query)

test_vector = test_dictionary.doc2bow(pro_query)

test_vector_tfidf = test_tfidf[test_vector]

sims = test_docsim_index[test_vector_tfidf]

for i, score in sims:
  print(ebay_responses[i], score)


 Hi there. Sorry to hear the item has not arrived. Is there tracking for the item? If not, I'd advise opening an item not received claim. You can learn more about that here&gt; https://t.co/0toc8cKSB0. Any other questions, let us know. ^SG 0.9377691149711609
 Need to contact customer service directly. Please DM contact email for customer service 0.6280146837234497
 0.448180615901947
 0.3625844419002533
 As the seller is responsible for the courier, they can claim for lost items from the courier. You still have to open a claim through eBay to be protected. Please follow the previous steps. We cant outbound DM. You can DM us if you have any other questions. ^SG 0.27123868465423584


/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))


'''
@115938 Hi there. Sorry to hear the item has not arrived. Is there tracking for the item? If not, I'd advise opening an item not received claim. You can learn more about that here&gt; https://t.co/0toc8cKSB0. Any other questions, let us know. ^SG 0.9377691149711609

@AskeBay Need to contact customer service directly. Please DM contact email for customer service 0.6280146837234497

 0.448180615901947
 0.3625844419002533
@115938 As the seller is responsible for the courier, they can claim for lost items from the courier. You still have to open a claim through eBay to be protected. Please follow the previous steps. We cant outbound DM. You can DM us if you have any other questions. ^SG 0.27123868465423584



In [0]:
print(ebay_questions[8])

print(test_questions[8])

print(test_responses[8])


115938-@AskeBay need assistance. Bought item, due to be delivered Saturday. Never received item, despite being marked as delivered. No response from seller, delivery services cannot assist with lost/missing package. Please advise.
['need', 'assistance', 'buy', 'item', 'due', 'deliver', 'saturday', 'never', 'receive', 'item', 'despite', 'mark', 'deliver', 'response', 'seller', 'delivery', 'service', 'assist', 'lostmissing', 'package', 'please', 'advise']
 Hi there. Sorry to hear the item has not arrived. Is there tracking for the item? If not, I'd advise opening an item not received claim. You can learn more about that here&gt; https://t.co/0toc8cKSB0. Any other questions, let us know. ^SG


# Build the mode with entire corpus

In [0]:
dictionary = Dictionary(processed_ebay_questions)

tfidf = TfidfModel(dictionary = dictionary)

word2vec_model = Word2Vec(processed_ebay_questions
                     , workers=cpu_count()
                     , min_count=5
                     , size=300
                     , seed=12345)

sim_index = WordEmbeddingSimilarityIndex(word2vec_model.wv)

sim_matrix = SparseTermSimilarityMatrix(sim_index
                                               , dictionary
                                               , tfidf
                                               , nonzero_limit=100)

bow_corpus = [dictionary.doc2bow(document) for document in processed_ebay_questions]

tfidf_corpus = [tfidf[bow] for bow in bow_corpus]

docsim_index = SoftCosineSimilarity(tfidf_corpus, sim_matrix, num_best=10)


# Make an example

### Example 1

In [0]:
query1 = "I need an assistance. The item is marked delivery but I never receive it"

_query1 = preprocessing(query1)

vector1 = dictionary.doc2bow(_query1)

vector1_tfidf = tfidf[vector1]

sims1 = docsim_index[vector1_tfidf]

for i, score in sims1:
  print(ebay_responses[i], score)

 Eek! As sellers may ship at any time, we leave it up to them to cancel. Keep in contact with them to clarify what's happening. ^B 1.0
 1.0
 1.0
 Hi! I know it's important to get this money. Could you DM us with your PayPal email and a screenshot of the funds? Thanks! ^ES 1.0
 It probably will go out soon. We do try to get it done in 1 business day, but we appreciate the feedback. ^BL 1.0
 1.0
 1.0
 That is odd. Was tracking uploaded in the system? ^B 1.0
 Thanks for the update on this. Sounds like they're still on top of it, correct? Keep us posted. ^CC 1.0
 1.0


/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))


- If you do run into this and the fee isn't credited, just DM us the details and we'll be happy to take a look! ^B 1.0
- We'll take a look! DM us your item #, email address, first/last name and postal code on file with more information. ^B 1.0
- I'm sorry to hear you're having these troubles. You should be able to print labels from the app itself. Are you not getting the option when you view the purchase? ^BL 1.0
- If an item isn't as described a return may still be needed and we would help with that, but you can see https://t.co/QDgDg5dz42 if you think it's intentional so we can ensure it's not misused. Keep us posted! ^D 1.0
- You’re welcome to relist now, just make sure to end the listing if the buyer ends up paying. Once you open the unpaid item case you can close it on the 4th day! ^JN 1.0
 1.0
- Hi there! Keep in mind, once a case is closed you can't reopen it so you wouldn't be able to ask for a refund for the replacement.^C 1.0
 1.0
- Apologies if the way I worded things upset you. I can definitely pass your concerns on to my manager but the best course of action is to submit your suggestion to the link I provided so the relevant team gets it and can review it for possible implementation. ^V 1.0
- Sorry about that! When did you turn them off? It may take a bit. You may also want to toggle all notifications off, then on. ^S 1.0
/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))

### Example 2

In [0]:
query3 = "Please repond to DM"
_query3 = preprocessing(query3)

vector3 = dictionary.doc2bow(_query3)

vectors_tfidf = tfidf[vector3]

sims3 = docsim_index[vectors_tfidf]

for i, score in sims3:
  print(processed_ebay_questions[i], ebay_responses[i], score)

['get', 'back', 'three', 'time', 'yesterday', 'take', 'hour', 'respond', 'today', 'everyone', 'vacation', 'already']  We're sorry it felt that way! We work to answer messages as quickly as possible. We've just replied to your DM. Let us know if you have other questions for now. ^LB 1.0
['open', 'request', 'must', 'unsuspended', 'keep', 'upfated', 'thank', 'team', 'ebay']  No worries, Neil! We'll be here if you need us! ^B 1.0
['hello', 'please', 'check', 'msg', 'thank']  Thank you for your message. We have just replied to your DM with more information! ^JL 1.0
['anytime', 'day', 'actually', 'think', 'something', 'go', 'sell', 'jack', 'use', 'sell', 'alot', 'search', 'engine', 'attempt', 'compete', 'amazon', 'seem', 'would', 'say', 'go', 'back', 'previous', 'format', 'use', 'original', 'search', 'engine', 'cassinni']  1.0
['gsp', 'overpriced', 'inconsistent', 'kill', 'ability', 'shop', 'overseas', 'us', 'check', 'ship', 'small', 'empty', 'box', 'australia', 'need', 'fix', 'ebay', 'ebayg

/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))


['contact', 'several', 'time', 'amp', 'hit', 'brick', 'wall', 'try', 'arrange', 'collection', 'via', 'msg', 'bfr', 'purchase', 'ban', 'amp', 'lose', 'day', 'taking', 'httpstcoskxdicitw']  Hey there! We'd be happy to investigate further if you send us over a DM with details. Sorry if we've missed anything! ^CR 1.0
['glad', 'help', 'obstructive', 'rude', 'want', 'escalation', 'contact', 'detail', 'staff', 'refuse', 'give']  1.0

['product', 'undelivered', 'purchase', 'remove', 'system', 'unable', 'contact', 'support', 'seller', 'unresponsive', 'need', 'help']  We're here to help! Send us a DM with first/last name, email address, postal/zip code and info about the item and we'll review! ^CR 1.0

['dm', 'would', 'return', 'query', 'hope', 'help']  Hey we got it! We'll pick up the conversation over there! ^CR 1.0

['poor', 'service', 'restrict', 'high', 'volume', 'seller', 'discuss', 'transaction', 'via', 'msg', 'downhill', 'well', 'service']  We'd be happy to talk with you more about what caused the restriction. Feel free to DM us with more details and we'll review. ^CR 1.0

['need', 'change', 'numb', 'since', 'long', 'use', 'find', 'way', 'thati', 'trouble', 'list']  Hey Dewi, thanks for getting in touch. You can update that here: https://t.co/LL6QKfBFMr. Let us know if we can help. ^CM 1.0

['use', 'mobile', 'app', 'create', 'list', 'check', 'mobile', 'friendly', 'laptop', 'need', 'fix']  Hey Dennis. What do you mean that if you check the mobile-friendly on the laptop? Can you please send us a screenshot? Thanks ^A 1.0

['hey', 'ebay', 'kind', 'need', 'help', 'studiescan', 'give', 'docsinfos', 'datum', 'strategynext', 'move', 'ebay', 'thx', 'lot']  We love that you'll be finding out more about eBay! Check out https://t.co/pRy6K18nod for some great info. Best of luck w/ your studies! ^LB 1.0

['bid', 'restriction', 'help', 'find', 'restrict', 'remove', 'thank', 'lot', 'httpstcotyzndsmhk']  Sure! DM us your full name, registered email address, &amp; registered postal code. We're happy to help! ^AE 1.0

['yes', 'also', 'report', 'via', 'phone', 'tell', 'would', 'contact', 'response', 'seller']  Since the seller has been notified, it’s up to them to reach out to you if they’d like their item back. ^BT 1.0


In [0]:
print(ebay_questions[700])
print(ebay_responses[700])

168920-@AskeBay wait is a perfume bottle considered dangerous material?
 Looks like you can, if they don't contain alcohol: https://t.co/NBaDNpnGh6. Glad to hear you got it shipped out! ^TL


### Example 3

In [0]:
query4 = "I'm not sure if the bottle of perfume is considered as dangerous material?"
_query4 = preprocessing(query4)
vector4 = dictionary.doc2bow(_query4)
vector4_tfidf = tfidf[vector4]

sims4 = docsim_index[vector4_tfidf]

for i, score in sims4:
  print(processed_ebay_questions[i], ebay_responses[i], score)

['wait', 'perfume', 'bottle', 'consider', 'dangerous', 'material']  Looks like you can, if they don't contain alcohol: https://t.co/NBaDNpnGh6. Glad to hear you got it shipped out! ^TL 0.9667293429374695
['please', 'see', 'attach', 'field', 'camera', 'give', 'degree', 'angle', 'give', 'lense', 'would', 'almost', 'half', 'sphere', 'protrude', 'lens', 'camera', 'similar', 'one', 'picture', 'false', 'advertise', 'purchase', 'camera', 'httpstcolfgduoab']  0.8126991987228394
['light', 'travel', 'straight', 'line', 'light', 'enter', 'degree', 'center', 'degree', 'field', 'lens', 'protude', 'lot', 'camera', 'advertise', 'doe', 'half', 'sphere', 'lens', 'front', 'false', 'advertise']  Thanks for the details. As it seems to be a concern with the manufacturer specs and not the listings themselves, it does not sound like something we can action here. You will want to discuss that with the manufacturer to determine why they claim 170 instead. ^D 0.8126991391181946
['duty', 'keep', 'platform', 'cle

/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
/usr/local/lib/python3.6/dist-packages/gensim/similarities/termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))


['wait', 'perfume', 'bottle', 'consider', 'dangerous', 'material']  Looks like you can, if they don't contain alcohol: https://t.co/NBaDNpnGh6. Glad to hear you got it shipped out! ^TL 0.9667293429374695

['light', 'travel', 'straight', 'line', 'light', 'enter', 'degree', 'center', 'degree', 'field', 'lens', 'protude', 'lot', 'camera', 'advertise', 'doe', 'half', 'sphere', 'lens', 'front', 'false', 'advertise']  Thanks for the details. As it seems to be a concern with the manufacturer specs and not the listings themselves, it does not sound like something we can action here. You will want to discuss that with the manufacturer to determine why they claim 170 instead. ^D 0.812565803527832

['generate', 'fedex', 'label', 'overestimate', 'weight', 'come', 'much', 'light', 'adjust', 'price', 'thank', 'advance']  Yes! FedEx doesn't charge you until the item is delivered, so they'll charge you the correct amount based on the weight. ^BL 0.8121383190155029

['constitute', 'special', 'punctuation', 'mean', 'remove', 'normal', 'punctuation', 'like', 'full', 'stop', 'say', 'slash', 'constitute', 'swear', 'please', 'clarify']  It's difficult to identify for sure, but it looks likes the term "bit cheap" may not recognize the space there. I't try another word and see if that works! ^D 0.8120605945587158

['anytime', 'day', 'actually', 'think', 'something', 'go', 'sell', 'jack', 'use', 'sell', 'alot', 'search', 'engine', 'attempt', 'compete', 'amazon', 'seem', 'would', 'say', 'go', 'back', 'previous', 'format', 'use', 'original', 'search', 'engine', 'cassinni']  0.8106344938278198

['amigooo', 'necesito', 'datos', 'de', 'cuenta', 'bancaria', 'para', 'el', 'reintegro', 'es', 'estado', 'actual', 'del', 'producto', 'como', 'llegó', 'destino', 'que', 'patetico', 'ustedes', 'de', 'ebay', 'hay', 'justicia', 'para', 'los', 'vendedores', 'si', 'la', 'hay', 'retiro', 'de', 'sus', 'politicas', 'absurdas']  Hi Nino. Unfortunately our Spanish support isn't available for the rest of the day. Although we recommend that you go here https://t.co/o1PE5TGxBo after logging in to find the details. ^KE 0.7033706903457642
['imply', 'ebay', 'feel', 'control', 'fb', 'favor', 'buyer', 'increase', 'revenue', 'let', 'face', 'amazon', 'trump', 'ebayebay', 'decline', 'past', 'year', 'ebay', 'cold', 'amp', 'unappreciative', 'sellersamazon', 'cust', 'serv', 'leave', 'ebay', 'dust', 'light', 'year']  0.6342083811759949
['actually', 'week', 'miss', 'since', 'upset', 'barely', 'miss', 'coupon', 'three', 'hour', 'today', 'item', 'go', 'back', 'regular', 'price', 'also']  Sorry to hear that. I understand that would be tough, but hopefully you'll see another coupon/deal soon. ^BL 0.6339040994644165

['hello', 'touch', 'base', 'tomorrow', 'item', 'start', 'b', 'relisted', 'putt', 'thougt', 'issue', 'current', 'plan', 'action', 'order', 'upset', 'buyerscustomers', 'likely', 'affect', 'feedback', 'score', 'plan']  Following. Please understand Im doing this to put my customers at ease and protect the intergerty of my feedback. 1) for each item that the ebay computer adds a "Best Offer" to I will remove it.
2) I will also place a disclaimer in the listing letting customers know it was 0.6336686611175537

['ebay', 'list', 'sale', 'day', 'clearly', 'choose', 'general', 'problem', 'atm', 'ebayuk']  Are you listing on the site or app? It's possible that Good 'Til Cancelled is added to your fixed price items automatically. ^S 0.6332412362098694

